In [1]:
import os
import re
import shutil
import pandas as pd
from PIL import Image
from markdownify import MarkdownConverter

To_convert = {
    'Module':[],
    'Slides':[]
}

htmlFiles = []
path = '../../GEOS270_Archive/Modules/'
for mod in range (7):
    for dir,subdir,filename in os.walk(f'{path}{mod+1}'):
        if 'content' in dir and '_site' not in dir:
            for f in filename:
                if '.html' in f:
                    To_convert['Module'].append(mod+1)
                    To_convert['Slides'].append(f)

To_convert = pd.DataFrame(data=To_convert)
To_convert['Name'] = None

To_convert.loc[To_convert.index==0,'Name'] = 'History of GIS'
To_convert.loc[To_convert.index==1,'Name'] = 'Steps of Abstraction'
To_convert.loc[To_convert.index==2,'Name'] = 'Map Projections'
To_convert.loc[To_convert.index==4,'Name'] = 'Data Classification'
To_convert.loc[To_convert.index==5,'Name'] = 'Data Visualization'
To_convert.loc[To_convert.index==6,'Name'] = 'Phenomena and Data Types'
To_convert.loc[To_convert.index==10,'Name'] = 'Choice of Data Model'
To_convert.loc[To_convert.index==11,'Name'] = 'The Raster Data Model'
To_convert.loc[To_convert.index==12,'Name'] = 'Spatial is Special'
To_convert.loc[To_convert.index==13,'Name'] = 'The Vector Data Model'
To_convert.loc[To_convert.index==14,'Name'] = 'Spatial Data Workflows'
To_convert.loc[To_convert.index==15,'Name'] = 'Raster Analysis Methods'
To_convert.loc[To_convert.index==16,'Name'] = 'Vector Analysis Methods'
To_convert.loc[To_convert.index==18,'Name'] = 'Ethics in GIS'
To_convert.loc[To_convert.index==19,'Name'] = 'Spatial Sampling Methods'
To_convert.loc[To_convert.index==21,'Name'] = 'Uncertainty in GIS'

To_convert


,Module,Slides,Name
0,1,HistoyGIS.html,History of GIS
1,2,Abstraction.html,Steps of Abstraction
2,2,Projection.html,Map Projections
3,2,Web_Mercator_Iqualuit.html,None
4,3,Classification.html,Data Classification
5,3,Visualization.html,Data Visualization
6,3,WhatIsData.html,Phenomena and Data Types
7,4,Bldg.html,None
8,4,Choice.html,None
9,4,ChoiceOfModel.html,None


In [2]:
def head(theme='default'):
  return (f'''---
format:
  revealjs:
    theme: {theme}
    code-fold: true
    controls: true
    navigation-mode: linear
    controls-layout: bottom-right
    controls-tutorial: true
    slide-number: true
    show-slide-number: all
    pdfMaxPagesPerSlide: 1
fig-dpi: 300

---''')

In [3]:
def find_between(s, first, last):
    try:
        regex = rf'{first}(.*?){last}'
        return re.findall(regex, s)
    except ValueError:
        return -1


class ImageBlockConverter(MarkdownConverter):
    """
    Create a custom MarkdownConverter that adds two newlines after an image
    """
    def convert_img(self, el, text, convert_as_inline):
        return super().convert_img(el, text, convert_as_inline)+'\n\n'

class convert_Reveal():
    def __init__(self,dir,fileName,Title=None):
        self.html = f'{dir}{fileName}.html'
        # if Title == None:
        #     self.Name = fileName
        # else:
        #     self.Name = Title
        self.dest = f"{Title.replace(' ','_')}/"
        self.dir = dir
        self.parse_Slides()

    def md(self,html, **options):
        return ImageBlockConverter(**options).convert(html)
    
    def titleSlide(self):
        TTL = self.text_md.split('---')[2].split('==')[0]
        TTL_txt = TTL.split('\n')

        k = 0
        for i,t in enumerate(TTL_txt):
            if len(t)>1:
                k = i

        ttl = TTL_txt[k]
        New_Title = TTL.replace(ttl,Title)
        self.text_md.replace(TTL,New_Title)

    def parse_Slides(self):
        html = open(self.html, encoding="utf8").read()
        html = html.replace('text-align: left','text-align: Left')
        html = html.replace('text-align: center','text-align: Center')
        html = html.replace('text-align: right','text-align: Right')
        html = html.replace(' style="text-align: Left;"','')
        html = html.replace(' style="text-align: Center;"','')
        html = html.replace(' style="text-align: Right;"','')
        html = html.replace(' style="text-align: Left"','')
        html = html.replace(' style="text-align: Center"','')
        html = html.replace(' style="text-align: Right"','')
        html = html.replace('<div class="subtitle">','<div class="subtitle"><p>::::{.columns}\n:::{.column width="100%"}</p>')
        html = html.replace('<div class="question">','<div class="question"><p>::::{.columns}\n:::{.column width="100%"}</p>')
        html = html.replace('<div class="full">','<div class="full"><p>::::{.columns}\n:::{.column width="100%"}</p>')
        html = html.replace('<div class="left">','<div class="left"><p>::::{.columns}\n:::{.column width="50%"}</p>')
        html = html.replace('<div class="right">','<div class="right"><p>:::\n:::{.column width="50%"}</p>')
        html = html.replace('<div class="r-stack">','<div class="r-stack"><p>:::\n:::{.column width="50%"}</p>')
        html = html.replace('<div class="figure">','<div class="figure"><p>:::\n:::{.column width="50%"}</p>')
        html = html.replace('</section>','<p>:::\n::::</p></section>')
        slides = html.split('<div class="slides">')[-1].split('</div><!-- .slides -->')[0]
        theme = html.split('link id="theme"')[-1].split('/theme/')[-1].split('.css">')[0]
        self.YML_Header = head(theme)
        self.text_md = self.md(slides)



        with open(f'{self.dest}Slides.qmd','w+', encoding="utf8") as out:
            self.text_md = self.YML_Header+self.text_md
            out.write(self.text_md)
        self.move_Images()

    def move_Images(self,maxDim = 1800):
        # ims = self.md.split('![]')[1:]
        ims = self.text_md.split('![]')[1:]
        
        for im in ims:
            image_file = re.search(r'\((.*?)\)',im).group(1)
            if image_file.split('.')[-1].lower() in ['png','jpg','gif']:
                image = Image.open(self.dir+image_file)
                if image.mode != 'RGB' and image_file.split('.')[-1].lower()=='jpg':
                    image = image.convert('RGB')
                if image.size[0] > maxDim and image.size[1]<image.size[0]:
                    wpercent = (maxDim/float(image.size[0]))
                    hsize = int((float(image.size[1])*float(wpercent)))
                    image = image.resize((maxDim,hsize), Image.Resampling.LANCZOS)
                    image.save(f'{self.dest}{image_file}',optimize=True)
                elif image.size[1] > maxDim and image.size[1]>image.size[0]:
                    wpercent = (maxDim/float(image.size[1]))
                    wsize = int((float(image.size[0])*float(wpercent)))
                    image = image.resize((wsize,maxDim), Image.Resampling.LANCZOS)
                    image.save(f'{self.dest}{image_file}',quality=95,optimize=True)
                else:
                    image.save(f'{self.dest}{image_file}',quality=95,optimize=True)
            else:
                try:
                    shutil.copy(self.dir+image_file,f'{self.dest}{image_file}')
                except:
                    print('Could not find ',self.dir+image_file)
                    print('Proceeding without')
                    print()
                    pass

YML = '''project:
  type: website
  output-dir: docs
  
format:
  html:
    theme: cosmo
    css: styles.css
    toc: true

website:
  title: "Lecture Slides"
  sidebar:
    style: "docked"
    search: true
    contents:
    
'''

for m in range(1,7):
    YML+=(f'      - section: "Module {m}"')
    YML+='\n'
    YML+=('        contents:')
    YML+='\n'
    for i ,row in To_convert[To_convert['Module']==m].iterrows():
        if row['Name'] is not None:
            Nm = row['Name'].replace(' ','_')        
            YML+=(f'        - {Nm}/Slides.qmd')
            YML+='\n'
            print('Converting: ',row['Name'])

            mod = To_convert['Module'][i]
            fileName = To_convert['Slides'][i].split('.')[0]
            Title = To_convert['Name'][i]

            dir = Title.replace(' ','_')
            if os.path.isdir(dir) == False:
                os.mkdir(dir)
            if os.path.isdir(dir+'/images') == False:
                os.mkdir(dir+'/images')

            conv = convert_Reveal(f'{path}{mod}/docs/content/',fileName,Title)
            
with open(f'_quarto.yml','w+', encoding="utf8") as out:
  out.write(YML)

Converting:  History of GIS
Converting:  Steps of Abstraction
Converting:  Map Projections
Converting:  Data Classification
Converting:  Data Visualization
Converting:  Phenomena and Data Types
Converting:  Choice of Data Model
Converting:  The Raster Data Model
Converting:  Spatial is Special
Converting:  The Vector Data Model
Converting:  Spatial Data Workflows
Converting:  Raster Analysis Methods
Converting:  Vector Analysis Methods
Converting:  Ethics in GIS
Converting:  Spatial Sampling Methods
Converting:  Uncertainty in GIS
